In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sciris as sc
import meso_tools as mt
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np
import os

In [ ]:
lims_credential_file = r"F:/Code/lims_credentials.dict"
lims_credentials = sc.loadobj(lims_credential_file)
max_proj_folder = r'\\allen\programs\mindscope\workgroups\learning\SNR\last_1000_frames'

### enter your mouse id here: 

In [ ]:
mouse = 629294

### This will regenerate max int projections from motion corrected stacks where they don't exist 

In [ ]:
lapi = mt.LimsApi(lims_credentials)
# get list of sessions/experiments for given mouse:
mouse_table = lapi.get_sessions_per_mouse_id(mouse)

if len(mouse_table) == 0:
    print(f"This mouse has no containers assigned yet")
else:

    # containers for this mouse:
    containers = list(mouse_table.container_id.drop_duplicates().values)

    # find container with largest number of sessions, get list of sessions from it:
    num_sessions_max = 0
    sessions_max = mouse_table.session_id[mouse_table.container_id==containers[0]].drop_duplicates()
    for container in containers:
        num_sessions = len(mouse_table.session_id[mouse_table.container_id==container].drop_duplicates())
        if num_sessions > num_sessions_max:
            num_sessions_max = num_sessions
            sessions_max = mouse_table.session_id[mouse_table.container_id==container].drop_duplicates()

    print(f"max number of sessions per container for this mouse is {num_sessions_max}")

    # make sure max projections exist for all experiments in max_proj_folder
    experiments = mouse_table.exp_id.values

    for exp in experiments: 

        write_path_mean = os.path.join(max_proj_folder, f'{exp}_avg_int.tif')
        write_path_max = os.path.join(max_proj_folder, f'{exp}_max_int.tif')
        path_mean_proj_offset = os.path.join(max_proj_folder, f'{exp}_avg_int_offset.tif')
        path_max_proj_offset = os.path.join(max_proj_folder, f'{exp}_max_int_offset.tif')

        mean_file = not os.path.exists(write_path_mean)
        max_file = not os.path.exists(write_path_max)
        mean_offset_file = not os.path.exists(path_mean_proj_offset)
        max_offset_file = not os.path.exists(path_max_proj_offset)

        if mean_file or max_file or mean_offset_file or max_offset_file:
            mc_stack_path = lapi.get_motion_corrected_stack(exp)

            if  mc_stack_path:

                #read motion corrected stack:
                mc_stack_path_local = mc_stack_path.replace('/', '\\')
                mc_stack_path_local = mc_stack_path_local.replace('\\allen', '\\\\allen')
                stack = mt.load_motion_corrected_movie(mc_stack_path_local, pages=-1000)

                # mean
                stack_mean = stack.mean(axis=0)
                stack_mean = stack_mean.astype(np.int16)
                mt.write_tiff(write_path_mean, stack_mean)

                # max
                stack_max = stack.max(axis=0)
                stack_max = stack_max.astype(np.int16)
                mt.write_tiff(write_path_max, stack_max)

                # zero-offset mean
                im_mean_offset = mt.offset_to_zero(stack_mean)
                im_mean_offset = im_mean_offset.astype(np.int16)
                mt.write_tiff(path_mean_proj_offset, im_mean_offset)

                # zero-offset max
                im_max_offset = mt.offset_to_zero(stack_max)
                im_max_offset = im_max_offset.astype(np.int16)
                mt.write_tiff(path_max_proj_offset, im_max_offset)

### check if any experiments shows up in multiple containers

In [ ]:
exps_repeating = []
exps = mouse_table.exp_id.drop_duplicates().values
for exp in exps:
    repeat = 0
    for container in containers: 
        container_exps = mouse_table.exp_id[mouse_table.container_id==container].values
        if exp in container_exps:
            repeat +=1
    if repeat != 1:
        exps_repeating.append(exp)
        print(f"Experiment {exp} repeats in more than one container")
        
print(f"\nTotal number fo experimenst is {len(exps)}, {len(exps_repeating)} assigned more than once to a container")

### plot mouse data using max intensity projections 

In [ ]:
# find max to use for normalized display of max projections
v_max = 0
for exp in mouse_table.exp_id.values:
    mean_proj_path = os.path.join(max_proj_folder, f'{exp}_max_int_offset.tif')
    im = mt.read_tiff(mean_proj_path)
    im_max = np.max(im)
    if im_max > v_max:
        v_max=im_max

In [ ]:
# plot containers using max projections: 
for container in containers: 
    i=1
    fig = pl.figure(figsize = [len(sessions_max)*10, 10])
    for session in sessions_max: 

        exp = mouse_table.exp_id[mouse_table.container_id==container][mouse_table.session_id==session]
        if len(exp.values) != 0:
            if len(exp) == 1:
                exp=int(exp.values)
            else:
                exp=int(exp.values[0])

            max_proj_path = os.path.join(max_proj_folder, f'{exp}_max_int_offset.tif')
            if os.path.exists(max_proj_path):
                im = mt.read_tiff(max_proj_path)
                snr = np.round(mt.compute_basic_snr(im),3)
                accutance = np.round(mt.compute_acutance(im),3)
                photon_flux = np.round(np.sqrt(np.mean(im.flatten())),1)
                ax = fig.add_subplot(1, num_sessions_max, i)
                ax.imshow(im, cmap='jet', vmin=0, vmax = v_max)
                ax.set_title(f"ses {session}, exp {exp}\nSNR {snr} ph_flux {photon_flux}\n accutance {accutance}", fontsize=22)
                ax.axis('off')
                i+=1
            else:
                ax = fig.add_subplot(1, num_sessions_max, i)
                ax.axis('off')
                i+=1
        else:
            ax = fig.add_subplot(1, num_sessions_max, i)
            ax.axis('off')
            i+=1

    fig.text(0.1, 0.45, f"container\n{container}", fontsize=22, rotation=90)

### plot mouse data using average int projections 

In [ ]:
# find max to use for normalized display of max projections
v_max = 0
for exp in mouse_table.exp_id.values:
    mean_proj_path = os.path.join(max_proj_folder, f'{exp}_avg_int_offset.tif')
    im = mt.read_tiff(mean_proj_path)
    im_max = np.max(im)
    if im_max > v_max:
        v_max=im_max

# plot containers using avg projections: 
for container in containers: 
    i=1
    fig = pl.figure(figsize = [len(sessions_max)*10, 10])
    for session in sessions_max:

        exp = mouse_table.exp_id[mouse_table.container_id==container][mouse_table.session_id==session]
        if len(exp.values) != 0:
            if len(exp) == 1:
                exp=int(exp.values)
            else:
                exp=int(exp.values[0])

            max_proj_path = os.path.join(max_proj_folder, f'{exp}_avg_int_offset.tif')
            if os.path.exists(max_proj_path):
                im = mt.read_tiff(max_proj_path)
                snr = np.round(mt.compute_basic_snr(im),3)
                accutance = np.round(mt.compute_acutance(im),3)
                photon_flux = np.round(np.sqrt(np.mean(im.flatten())),1)
                ax = fig.add_subplot(1, num_sessions_max, i)
                ax.imshow(im, cmap='jet', vmin=0, vmax = v_max)
                imaging_quality = np.round(photon_flux * accutance, 1)
                ax.set_title(f"ses {session}, exp {exp}\nImaging quality : {imaging_quality}", fontsize=22)
                ax.axis('off')
                
                if exp in exps_repeating : ## I ADDED THIS
                    autoAxis = ax.axis()
                    rec = pl.Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=7,color='r')
                    rec = ax.add_patch(rec)
                    rec.set_clip_on(False)
                
                i+=1
            else:
                ax = fig.add_subplot(1, num_sessions_max, i)
                ax.set_title(f"ses {session}, exp {exp}", fontsize=22)
                ax.axis('off')
                i+=1
        else:
            ax = fig.add_subplot(1, num_sessions_max, i)
            ax.axis('off')
            i+=1

    fig.text(0.1, 0.45, f"container\n{container}", fontsize=22, rotation=90)